In [ ]:
!pip install torch


In [ ]:
!pip install transformers peft huggingface_hub

In [ ]:
!pip install peft deepspeed optimum accelerate

In [ ]:
!pip install -U bitsandbytes

In [1]:
import torch
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
else:
    print("No GPU available")

NVIDIA L4


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from huggingface_hub import login

login(token="")

/home/karthick/finetuning/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/karthick/.cache/huggingface/token
Login successful


In [3]:
base_model_name = "Qwen/Qwen2-1.5B"
adapter_model_name = "tryhighlight/qwen2-1.5B-ocr-task-detection"

config = PeftConfig.from_pretrained(adapter_model_name)

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )
)

In [5]:
model = PeftModel.from_pretrained(model, adapter_model_name)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print(tokenizer.special_tokens_map)

In [7]:
model.eval()  # Set the model to evaluation mode

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4

In [8]:
def apply_chat_template(system_prompt, user_name, ocr_text, max_len):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "My name is " + user_name + " \n" + ocr_text},
    ]
    tokenized_output = tokenizer.apply_chat_template(
                            messages,
                            tokenize=True,
                            add_generation_prompt=False,
                            padding=False,
                            max_length=max_len,
                            truncation=True,
                    )
    input_ids = torch.tensor(tokenized_output, dtype=torch.int)
    print (input_ids.shape)
    return input_ids

def generate_response(system_prompt, user_name, ocr_text):
    max_len = 4096
    input_ids = apply_chat_template(system_prompt, user_name, ocr_text, max_len)
    input_ids = input_ids.to(model.device)
    # Ensure input_ids is 2D
    if input_ids.dim() == 1:
        input_ids = input_ids.unsqueeze(0)
    # Create an attention mask for the input_ids
    attention_mask = torch.ones_like(input_ids, device=model.device)

    # Check dimensions
    print("Input IDs Shape:", input_ids.shape)
    print("Attention Mask Shape:", attention_mask.shape)

    outputs = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=50, pad_token_id=tokenizer.pad_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [9]:
# Example usage
system_prompt = "You are a helpful AI assistant. User will provide full name followed by the OCR content of his/her computer screen. Looking at the OCR, first detect if there are any email or messaging or any other kind of conversations in it. If yes, then detect if there are any TODOs that the above user has to complete as a result of the conversation. If yes, just provide a short single line task that can be directly added to the todo list. If there is no conversation detected or no task detected in the conversation as a TODO, just output the exact phrase 'No task'."

In [38]:
user_name = "@Karthick"

In [34]:
ocr_text = "8 hhightignt (Chane -Medal-s' x |) | | | I | | I |\nΓé¼ > G@ 8% appslack.com/client/T037W748A/CO6ADHR1G3F?cdn_fallback=1 * O10:\n& ┬⌐ Search Medal Q @\nMedal v = Gi @highlight ┬⌐ _ bttos:/app.amplitude.com/analytics/medal/chart/new/fmitanwb BBB x 63┬╗ Weamas\n@Atharva what's the latest on the non serverless endpoints so the warm up time is better - and what's up with handling lang context? The APIs shoul \= nati ~┬Ñ this nd users should be informed.\ne ┬⌐ Threads @Josh / @Sam when will screenshots work normally in HL Chat? Do we need to do more designs to unblock team next week? What work do we nee. Mond┬Ñ-July 15th ~ cht chat to a great UX quickly?\nE> Drafts & sent @Maximilian Maksutovic when can we integrate conversations with floaty? What's blocking it from being able to open from attachments? How are we going to fit it into onboarding? These questions all need answers and progress towards\nHome @Mark what's holding us back from more marketing channels? (edited)\na ~ External Connections BD BB 2 eeviies Last realy 3 days ago |\nMs ##-_ medal-akamai @ Pim 74sane\n) _* shated-medat-honeycomb @lulian i had a note from last week that you'd ship the auth skipping - did that happen last week?\npetty ΓÇö-┬Ñ Channels BBD ┬╗ vies Last ents das aso\nOW sees a Pim 10:21am\nQ dt beckend replied to a thread: IMG_1200\nLater @lulian *\n# builds 2 @\n1 Γé¼\n4 chatgpt\nsts 4 community g Sukkrit 6:37 eM\noo <M Hiteam, down with fever, will rest up for the day\n- weed New\nvon fF eacombesuer - Canna:\n# data-and-experiments ┬« Brandon ΓÇ£> 6:26 pm\n3D Tried using Highlight Chat on my windows this weekend and it just responded to me with an empty message. I'l see if I can reproduce it\n4 desktop-frontend\na me\n# desktop-github\nper BB 2 enea\n& highlight a Pim 7.03PM\n# incidents replied to a thread: \"Cmd .\" And now highlight is frozen\nA * @Karthick can we get to the bottom of this?\n┬⌐ incoming-medal-\nΓÇö e\n4 Issue-tracking .\new newer replies\n& leaders\n& Pim 7:10PM\na ED replied to a thread: \"Cmd.:\" And now highlight is frozen\n# move we already are not any new users - the least we can do is make sure our existing users aren't churning due to crahses, gotta prioritize this\n# nyclife\n# random\n+ essage @highlight\n# releases ┬░\nfA #  sentry-alerts tha O@ Ge OB\nLA it ser ┬Ñ More unreads\n"

In [39]:
response = generate_response(system_prompt, user_name, ocr_text)
print("Generated response:", response)

torch.Size([867])
Input IDs Shape: torch.Size([1, 867])
Attention Mask Shape: torch.Size([1, 867])
Generated response: system
You are a helpful ai assistant. User will provide full name followed by the OCR content of his/her computer screen. Looking at the OCR, first detect if there are any email or messaging or any other kind of conversations in it. If yes, then detect if there are any TODOs that the above user has to complete as a result of the conversation. If yes, just provide a short single line task that can be directly added to the todo list. If there is no converstaion detected or no task detected in the conversation as a TODO, just output the exact phrase "No task".
user
My name is @Karthick 
8 hhightignt (Chane -Medal-s' x |) | | | I | | I |
Γé¼ > G@ 8% appslack.com/client/T037W748A/CO6ADHR1G3F?cdn_fallback=1 * O10:
& ┬⌐ Search Medal Q @
Medal v = Gi @highlight ┬⌐ _ bttos:/app.amplitude.com/analytics/medal/chart/new/fmitanwb BBB x 63┬╗ Weamas
@Atharva what's the latest on the

In [10]:
import transformers
from datasets import load_dataset
from typing import Dict, List, Optional

max_len = 4096

dataset_name = "tryhighlight/ocr_task_dataset"
#Importing the dataset
input_dataset = load_dataset(dataset_name, split="all")
print (input_dataset)

Dataset({
    features: ['NAME', 'OCR', 'TASK'],
    num_rows: 77568
})


In [ ]:
no_task = 0
for row in input_dataset:
    if row["TASK"] == "No task":
        no_task += 1
print (no_task)

In [11]:
test = input_dataset.shuffle(seed=65).select(range(100))
for row in test:
    task_prefix = "Task : " if row["TASK"] != "No task" else ""
    task = task_prefix + row["TASK"]
    print (task)

No task
No task
No task
Task : Check voice mail for a message from Ed regarding the storage.
No task
No task
No task
No task
No task
No task
No task
No task
No task
No task
No task
No task
No task
No task
No task
No task
Task : Set up Monarch Steel Company in global system.
No task
Task : Provide important dates to MAWhitt@aol.com
Task : Process and approve or reject outstanding business expenses in iPayIt and XMS by 2:00 p.m. Monday, December 3, 2001.
Task : No task detected.
Task : Approve access request for gulay.soykok@enron.com.
No task
No task
Task : Complete practice questions for Crack Spread, BTU, and Spark Spread.
Task : Resend the email version which Salt River sent to you
No task
No task
No task
Task : Approve access request for erik.simpson@enron.com for Market Data Kobra Power.
No task
No task
No task
No task
Task : Complete self assessment in PAS by Friday, May 17, 2002.
No task
Task : Determine the demand charges for path #17 and #21.
No task
No task
Task : Forward the 

In [17]:


# Get the size of the dataset
dataset_size = len(input_dataset)
print(f"Size of the dataset: {dataset_size} samples")
system_prompt = "You are a helpful ai assistant. User will provide full name followed by the OCR content of his/her computer screen. Looking at the OCR, first detect if there are any email or messaging or any other kind of conversations in it. If yes, then detect if there are any TODOs that the above user has to complete as a result of the conversation. If yes, just provide a short single line task that can be directly added to the todo list. If there is no converstaion detected or no task detected in the conversation as a TODO, just output the exact phrase \"No task\"."
#    dataset = dataset.shuffle(seed=65).select(range(10000)) # Only use 1000 samples for quick demo
def check_length(row):
    # This function checks if the tokenized length is within the max length allowed
    input_content = tokenizer.encode(system_prompt + row["NAME"] + row["OCR"] + row["TASK"],
                                        add_special_tokens=True,
                                        truncation=False,
                                        return_length=True,
                                        max_length=None)
    return len(input_content) <= max_len - 20 # extra 20 tokens for the chat template

# Filter the dataset to exclude entries that are too long
filtered_dataset = input_dataset.filter(check_length)
print(f"Size of the dataset after filtering: {len(filtered_dataset)} samples")


Size of the dataset: 77568 samples


Filter: 100%|███████████████████████████████████████████████████████████| 77568/77568 [03:18<00:00, 390.28 examples/s]

Size of the dataset after filtering: 77543 samples


In [27]:
def format_chat_template(row):
    row_json = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "My name is " + row["NAME"] + " \n" + row["OCR"]}]
    tokenized_output = tokenizer.apply_chat_template(
                            row_json,
                            tokenize=True,
                            add_generation_prompt=False,
                            padding=False,
                            max_length=max_len,
                            truncation=True,
                    )
    input_ids = torch.tensor(tokenized_output, dtype=torch.int)

    return dict(
                input_ids=input_ids
            )

dataset = filtered_dataset.map(
    format_chat_template,
    num_proc=8,
)
print("Column names in the dataset:", dataset.column_names)

dataset = dataset.train_test_split(test_size=0.1, seed = 42)

Column names in the dataset: ['NAME', 'OCR', 'TASK', 'input_ids']


In [28]:
dataset["train"]

Dataset({
    features: ['NAME', 'OCR', 'TASK', 'input_ids'],
    num_rows: 69788
})

In [29]:
eval_dataset=dataset["test"]

In [30]:
eval_dataset = eval_dataset.shuffle(seed=65).select(range(100))

In [31]:
eval_dataset

Dataset({
    features: ['NAME', 'OCR', 'TASK', 'input_ids'],
    num_rows: 100
})

In [ ]:
eval_dataset[0]['input_ids']

In [32]:
for row in eval_dataset:
    input_ids = row['input_ids']
    input_ids = torch.tensor(input_ids, dtype=torch.int)
    input_ids = input_ids.to(model.device)
    # Ensure input_ids is 2D
    if input_ids.dim() == 1:
        input_ids = input_ids.unsqueeze(0)
    # Create an attention mask for the input_ids
    attention_mask = torch.ones_like(input_ids, device=model.device)

    outputs = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=50, pad_token_id=tokenizer.pad_token_id)
    outputs[input_ids.shape[1]:]
    text_output = tokenizer.decode(outputs[0][input_ids.shape[1]:], skip_special_tokens=True)
    print ('Expected output : ', row['TASK'])
    cleaned_output = text_output.replace("assistant\n", "")
    print ('Inference output : ', cleaned_output)

Expected output :  No task
Inference output :  No task

Expected output :  No task
Inference output :  No task

Expected output :  No task
Inference output :  No task

Expected output :  No task
Inference output :  No task

Expected output :  No task
Inference output :  No task

Expected output :  Attend the video conference on January 3, 2002.
Inference output :  No task

Expected output :  No task
Inference output :  No task

Expected output :  No task
Inference output :  No task

Expected output :  No task
Inference output :  No task

Expected output :  No task
Inference output :  No task

Expected output :  No task
Inference output :  No task

Expected output :  No task
Inference output :  No task

Expected output :  Confirm the FUGG imbalances and clarify their status.
Inference output :  Task : Confirm the correct imbalances on the G/L for FUGG and let Rita know what the true imbalances should be.

Expected output :  No task
Inference output :  Task : Please let Stacey Richardson

In [ ]:
token_text = tokenizer.decode(200)
token_text

In [ ]:
print (input_ids)

In [ ]:
print (attention_mask)

In [ ]:
eos_indices = (input_ids == tokenizer.eos_token_id).nonzero(as_tuple=True)[0]
len(eos_indices)

In [ ]:
eos_indices[0]

In [ ]:
first_eos_index = eos_indices[0]
labels[first_eos_index] = tokenizer.eos_token_id

In [ ]:
labels

In [ ]:
attention_mask[first_eos_index] = 1

In [ ]:
print (attention_mask)

In [ ]:
!python -V